In [6]:
# imports
import torch
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init

from torch.nn import Linear, Conv2d, BatchNorm2d, MaxPool2d, Dropout
from torch.nn.functional import relu, elu, relu6, sigmoid, tanh, softmax
from sklearn.metrics import accuracy_score
import torch.optim as optim

In [8]:
# Define model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.L = 500
        self.D = 128
        self.K = 1
        self.M = 64 
        ## feature extraction 
        self.featureExtract = nn.Sequential(nn.Conv1d(in_channels = 1 , out_channels = 32, kernel_size=8, padding =1, stride = 1),
                                            nn.LeakyReLU(negative_slope=0.2), 
                                            nn.MaxPool1d(2),
                                            nn.Conv1d(in_channels = 32,out_channels = 32*32, kernel_size=8, padding =1, stride = 1),
                                            nn.LeakyReLU(negative_slope=0.2),
                                            nn.MaxPool1d(2),
                                            nn.Conv1d(in_channels = 32*32, out_channels = 32*32*16, kernel_size=3, padding =1, stride = 1),
                                            nn.LeakyReLU(negative_slope=0.2),
                                            nn.MaxPool1d(2),
                                            nn.Conv1d(in_channels = 10, out_channels = 16, kernel_size=3, padding =1, stride = 1), 
                                            nn.LeakyReLU(negative_slope=0.2),
                                            nn.MaxPool1d(2),
                                            nn.Linear(320, self.M)) 

        self.attention = nn.Sequential(nn.Linear(self.L, self.D),
                                       nn.Tanh(),
                                       nn.Linear(self.D, self.K))
                
        self.classify = nn.Sequential(nn.Linear(self.M,32), 
                                      nn.LeakyReLU(negative_slope=0.2),
                                      Dropout(p = 0.2, inplace = False), 
                                      nn.Linear(32,16),
                                      nn.LeakyReLU(negative_slope=0.2),
                                      Dropout(p = 0.2, inplace = False), 
                                      nn.Linear(16,2), )
    
    def forward(self, x):
        ## feature extraction
        H = self.featureExtract(x)
        A = self.attention(H)
        A = torch.transpose(A,1,0)
        A = nn.softmax(A, dim = 1)
        M = torch.mm(A,H)
                         
            
        y_p = self.classify(M)
        y_hat = torch.ge(y_p,0.5).float()
        
        
        return x

    
net = Net()
print(net)    

Net(
  (featureExtract): Sequential(
    (0): Conv1d(500, 32, kernel_size=(8,), stride=(1,), padding=(1,))
    (1): LeakyReLU(negative_slope=0.2)
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(32, 32, kernel_size=(8,), stride=(1,), padding=(1,))
    (4): LeakyReLU(negative_slope=0.2)
    (5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv1d(10, 16, kernel_size=(3,), stride=(1,), padding=(1,))
    (7): LeakyReLU(negative_slope=0.2)
    (8): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv1d(10, 16, kernel_size=(3,), stride=(1,), padding=(1,))
    (10): LeakyReLU(negative_slope=0.2)
    (11): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Linear(in_features=320, out_features=64, bias=True)
  )
  (attention): Sequential(
    (0): Linear(in_features=500, out_features=128, bias=True)
    (1): Tanh()
    (2): Linear(in_fe

In [ ]:
# after the model finish training, save and load
# torch.save(the_model, PATH)
# the_model = torch.load(PATH)
